### Module Importation and DataFrame Setup

In [1]:
# Reset the model for re-run
from tensorflow.keras import backend as K
K.clear_session()

# Import dependencies
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
import keras_tuner as kt
from imblearn.over_sampling import RandomOverSampler
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
# Read the CSV file from the Dataset folder into a Pandas DataFrame
dfSurvivals = pd.read_csv(
    Path("../CSV_OUTPUT/Base_Cleaned_DS_CSV.csv")
)

In [3]:
# Review the DataFrame
dfSurvivals.head()

,Passenger Id,Title,First Name,Last Name,Sex,Age,Sibling/Spouse Aboard,Parent/Children Aboard,Passenger Class,Fare,Embarkation Port,Survival Boat,Body Number,Survived,age_group,family_size
0,1,Miss,Elisabeth Walton,Allen,female,29.0000,0,0,1,211.3375,S,2,S,1,Adults,0
1,2,Master,Hudson Trevor,Allison,male,0.9167,1,2,1,151.5500,S,11,S,1,Children,3
2,3,Miss,Helen Loraine,Allison,female,2.0000,1,2,1,151.5500,S,DNS,BNR,0,Children,3
3,4,Mr,Hudson Joshua Creighton,Allison,male,30.0000,1,2,1,151.5500,S,DNS,135,0,Adults,3
4,5,Mrs,Hudson J C (Bessie Waldo Daniels),Allison,female,25.0000,1,2,1,151.5500,S,DNS,BNR,0,Adults,3


In [4]:
# Print DataFrame information to ensure no null values and check datatypes
dfSurvivals.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1308 entries, 0 to 1307
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Passenger Id            1308 non-null   int64  
 1   Title                   1308 non-null   object 
 2   First Name              1308 non-null   object 
 3   Last Name               1308 non-null   object 
 4   Sex                     1308 non-null   object 
 5   Age                     1308 non-null   float64
 6   Sibling/Spouse Aboard   1308 non-null   int64  
 7   Parent/Children Aboard  1308 non-null   int64  
 8   Passenger Class         1308 non-null   int64  
 9   Fare                    1308 non-null   float64
 10  Embarkation Port        1308 non-null   object 
 11  Survival Boat           1308 non-null   object 
 12  Body Number             1308 non-null   object 
 13  Survived                1308 non-null   int64  
 14  age_group               1308 non-null   

In [5]:
# Preprocessing for features which are currently objects
dfSurvivals = pd.get_dummies(dfSurvivals, columns=["Title", "Sex", "Embarkation Port", "age_group"])

In [6]:
# Drop columns
dfSurvivals = dfSurvivals.drop(['Passenger Id', 'First Name', 'Last Name', 'Survival Boat', 'Body Number'], axis=1)
dfSurvivals.head()

,Age,Sibling/Spouse Aboard,Parent/Children Aboard,Passenger Class,Fare,Survived,family_size,Title_ Capt,Title_ Col,Title_ Don,...,Title_ the Countess,Sex_female,Sex_male,Embarkation Port_C,Embarkation Port_Q,Embarkation Port_S,age_group_Adults,age_group_Children,age_group_Seniors,age_group_Teenagers
0,29.0000,0,0,1,211.3375,1,0,0,0,0,...,0,1,0,0,0,1,1,0,0,0
1,0.9167,1,2,1,151.5500,1,3,0,0,0,...,0,0,1,0,0,1,0,1,0,0
2,2.0000,1,2,1,151.5500,0,3,0,0,0,...,0,1,0,0,0,1,0,1,0,0
3,30.0000,1,2,1,151.5500,0,3,0,0,0,...,0,0,1,0,0,1,1,0,0,0
4,25.0000,1,2,1,151.5500,0,3,0,0,0,...,0,1,0,0,0,1,1,0,0,0


In [7]:
# Ensure all columns are numerical
dfSurvivals.dtypes

Age                       float64
Sibling/Spouse Aboard       int64
Parent/Children Aboard      int64
Passenger Class             int64
Fare                      float64
Survived                    int64
family_size                 int64
Title_ Capt                 uint8
Title_ Col                  uint8
Title_ Don                  uint8
Title_ Dona                 uint8
Title_ Dr                   uint8
Title_ Jonkheer             uint8
Title_ Lady                 uint8
Title_ Major                uint8
Title_ Master               uint8
Title_ Miss                 uint8
Title_ Mlle                 uint8
Title_ Mme                  uint8
Title_ Mr                   uint8
Title_ Mrs                  uint8
Title_ Ms                   uint8
Title_ Rev                  uint8
Title_ Sir                  uint8
Title_ the Countess         uint8
Sex_female                  uint8
Sex_male                    uint8
Embarkation Port_C          uint8
Embarkation Port_Q          uint8
Embarkation Po

### Oversample to Correct Imbalance in Dataset
#### Oversampling code example used from https://www.section.io/engineering-education/how-to-handle-imbalanced-data-in-deep-neural-networks/#building-the-deep-neural-network-model-using-an-imbalanced-dataset

In [8]:
# Count the number of data samples in each class
count_majority_class, count_minority_class = dfSurvivals.Survived.value_counts()

In [9]:
# Save majority and minority classes in new variables
# In Survived column 0=No and 1=Yes 

df_majority_class = dfSurvivals[dfSurvivals['Survived'] == 0]
df_minority_class = dfSurvivals[dfSurvivals['Survived'] == 1]

In [10]:
# Oversample the minority data
df_class_oversample = df_minority_class.sample(count_majority_class, replace=True)

In [11]:
# Concatenate the oversampled minority class with the majority class
df_balanced_os = pd.concat([df_class_oversample, df_majority_class], axis=0)

In [12]:
# Count number of data samples in new balanced dataset
print('Number of data samples after over-sampling:')
print(df_balanced_os.Survived.value_counts())

Number of data samples after over-sampling:
1    808
0    808
Name: Survived, dtype: int64


In [13]:
# Review the DataFrame
df_balanced_os.head()

,Age,Sibling/Spouse Aboard,Parent/Children Aboard,Passenger Class,Fare,Survived,family_size,Title_ Capt,Title_ Col,Title_ Don,...,Title_ the Countess,Sex_female,Sex_male,Embarkation Port_C,Embarkation Port_Q,Embarkation Port_S,age_group_Adults,age_group_Children,age_group_Seniors,age_group_Teenagers
1076,18.0,0,0,3,7.7500,1,0,0,0,0,...,0,1,0,0,1,0,0,0,0,1
1195,29.0,0,0,3,9.5000,1,0,0,0,0,...,0,0,1,0,0,1,1,0,0,0
1181,21.0,0,0,3,7.6500,1,0,0,0,0,...,0,1,0,0,0,1,1,0,0,0
1050,19.0,1,1,3,15.7417,1,2,0,0,0,...,0,1,0,1,0,0,1,0,0,0
6,63.0,1,0,1,77.9583,1,1,0,0,0,...,0,1,0,0,0,1,0,0,1,0


### Split the data into X and y and then into testing and training sets

In [14]:
# Split the data into X (features) and y (target)

# Set the y variable, which is the target
y = df_balanced_os['Survived']

# Set the X variable, which includes all features escept the target
X = df_balanced_os.drop(columns=['Survived'])

In [15]:
# Split into testing and training sets using train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [16]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

### Compile, Train, and Evaluate the Model

In [17]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

input_dim = input_dim = X_train.shape[1]

def create_model(hp):
    nn_model = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh','sigmoid'])

    # Allow kerastuner to decide number of neurons in first layer
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=10,
        step=2), activation=activation, input_dim=input_dim))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 2)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=10,
            step=2),
            activation=activation))
   
    # Add final layer with sigmoid activation function
    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy","Precision","Recall"])

    return nn_model

In [18]:
# Set up the kerastuner then allow it to search for best hyperparameters
tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=20,
    hyperband_iterations=2,
    overwrite=True)

# Initialize EarlyStopping callback to monitor the validation loss
# Training will stop if the validation loss doesn't improve for 3 consecutive epochs
early_stopping = EarlyStopping(monitor='val_loss', patience=3)

# Perform hyperparameter search
tuner.search(X_train_scaled, y_train, epochs=20, validation_data=(X_test_scaled, y_test), callbacks=[early_stopping])

Trial 60 Complete [00h 00m 04s]
val_accuracy: 0.8193069100379944

Best val_accuracy So Far: 0.8267326951026917
Total elapsed time: 00h 02m 29s
INFO:tensorflow:Oracle triggered exit


In [19]:
# Get the hyperparameters of the best model
best_hyper = tuner.get_best_hyperparameters(1)[0]
best_hyper.values

{'activation': 'tanh',
 'first_units': 7,
 'num_layers': 2,
 'units_0': 3,
 'units_1': 9,
 'tuner/epochs': 20,
 'tuner/initial_epoch': 7,
 'tuner/bracket': 2,
 'tuner/round': 2,
 'tuner/trial_id': '0013'}

In [20]:
# Get the best model
best_model = tuner.get_best_models(1)[0]
best_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 7)                 238       
                                                                 
 dense_1 (Dense)             (None, 3)                 24        
                                                                 
 dense_2 (Dense)             (None, 9)                 36        
                                                                 
 dense_3 (Dense)             (None, 1)                 10        
                                                                 
Total params: 308 (1.20 KB)
Trainable params: 308 (1.20 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [21]:
# Train the model
fit_model = best_model.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
38/38 [==============================] - 1s 1ms/step - loss: 0.4773 - accuracy: 0.7937 - precision: 0.8109 - recall: 0.7653
Epoch 2/100
38/38 [==============================] - 0s 1ms/step - loss: 0.4748 - accuracy: 0.7929 - precision: 0.8084 - recall: 0.7669
Epoch 3/100
38/38 [==============================] - 0s 1ms/step - loss: 0.4732 - accuracy: 0.7896 - precision: 0.8028 - recall: 0.7669
Epoch 4/100
38/38 [==============================] - 0s 1ms/step - loss: 0.4721 - accuracy: 0.7929 - precision: 0.8094 - recall: 0.7653
Epoch 5/100
38/38 [==============================] - 0s 1ms/step - loss: 0.4707 - accuracy: 0.7921 - precision: 0.8080 - recall: 0.7653
Epoch 6/100
38/38 [==============================] - 0s 1ms/step - loss: 0.4697 - accuracy: 0.7888 - precision: 0.8024 - recall: 0.7653
Epoch 7/100
38/38 [==============================] - 0s 1ms/step - loss: 0.4680 - accuracy: 0.7904 - precision: 0.8042 - recall: 0.7669
Epoch 8/100
38/38 [=============================

In [22]:
# Evaluate the model using the test data
model_loss, model_accuracy, model_precision, model_recall = best_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}, Precision: {model_precision}, Recall: {model_recall}")

13/13 - 0s - loss: 0.4255 - accuracy: 0.8317 - precision: 0.8771 - recall: 0.7734 - 243ms/epoch - 19ms/step
Loss: 0.42546987533569336, Accuracy: 0.8316831588745117, Precision: 0.8770949840545654, Recall: 0.7733989953994751


In [23]:
# Use the model to make predictions on test data
y_pred_prob = best_model.predict(X_test_scaled).flatten()
y_pred = [1 if prob >= 0.5 else 0 for prob in y_pred_prob]

# Generate a confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

# Generate a classification report
class_report = classification_report(y_test, y_pred)
print("Classification Report:")
print(class_report)

13/13 [==============================] - 0s 917us/step
Confusion Matrix:
[[179  22]
 [ 46 157]]
Classification Report:
              precision    recall  f1-score   support

           0       0.80      0.89      0.84       201
           1       0.88      0.77      0.82       203

    accuracy                           0.83       404
   macro avg       0.84      0.83      0.83       404
weighted avg       0.84      0.83      0.83       404



In [25]:
# Export our model to keras file
best_model.save('OUTPUT/Titanic_survival_mod2_reprocessed_opt2.keras')